In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import deque
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.layers import Dense  # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore
import gym
import cv2
from threading import Thread
import tensorflow as tf
import time

def save_frame(frame_bgr, out):
    out.write(frame_bgr)

# Crear la red neuronal para aproximar Q(s, a) con arquitecturas variables
def build_model(state_dim, action_dim, layer_configs, learning_rate=0.001):
    model = Sequential()
    # Capa de entrada
    model.add(Dense(layer_configs[0], input_dim=state_dim, activation='relu'))
    # Capas ocultas
    for neurons in layer_configs[1:]:
        model.add(Dense(neurons, activation='relu'))
    # Capa de salida
    model.add(Dense(action_dim, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [2]:
class DQNAgent:
    def __init__(self, state_dim, action_dim, layer_configs, learning_rate=0.001, batch_size=64, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, replay_size=10000):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.replay_buffer = deque(maxlen=replay_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # Redes Q y Target con la arquitectura definida
        self.q_network = build_model(state_dim, action_dim, layer_configs, learning_rate)
        self.target_network = build_model(state_dim, action_dim, layer_configs, learning_rate)
        self.update_target_network()
    
    def update_target_network(self):
        """Copiar los pesos de la red principal a la red Target."""
        self.target_network.set_weights(self.q_network.get_weights())
    
    def select_action(self, state):
        """Selecciona una acción siguiendo la política epsilon-greedy."""
        if np.random.rand() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        state = np.expand_dims(state, axis=0)  # Añadir batch dimension
        q_values = self.q_network.predict(state, verbose=0)
        return np.argmax(q_values[0])
    
    def store_experience(self, state, action, reward, next_state, done):
        """Guarda una tupla de experiencia en el buffer."""
        self.replay_buffer.append((state, action, reward, next_state, done))
    
    def train(self):
        """Entrena la red Q usando un minibatch del buffer."""
        if len(self.replay_buffer) < self.batch_size:
            return
        
        # Muestreo aleatorio del replay buffer
        batch = random.sample(self.replay_buffer, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        
        states = np.array(states)
        next_states = np.array(next_states)
        rewards = np.array(rewards)
        dones = np.array(dones)
        
        # Valores actuales Q(s, a)
        q_values = self.q_network.predict(states, verbose=0)
        next_q_values = self.target_network.predict(next_states, verbose=0)
        
        # Actualizar valores Q usando la ecuación objetivo
        for i in range(self.batch_size):
            target = rewards[i]
            if not dones[i]:
                target += self.gamma * np.max(next_q_values[i])
            q_values[i][actions[i]] = target
        
        # Entrenar la red principal
        self.q_network.fit(states, q_values, epochs=1, verbose=0, batch_size=self.batch_size)


In [3]:
def train_dqn(env_name="MountainCar-v0", episodes=15, target_update_steps=100, layer_configs=[(64, 64)], learning_rate=0.001, gamma=0.99, batch_size=64):
    env = gym.make(env_name, render_mode="rgb_array")
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    
    agent = DQNAgent(state_dim, action_dim, layer_configs, learning_rate, batch_size, gamma)
    scores = []
    epsilons = []
    training_times = []
    
    state, _ = env.reset()
    sample_frame = env.render()
    if len(sample_frame.shape) == 2:
        sample_frame = np.expand_dims(sample_frame, axis=-1)
    height, width, channels = sample_frame.shape

    obj_video = cv2.VideoWriter_fourcc(*'mp4v')  
    out = cv2.VideoWriter("dqn_mountain_training_video.mp4", obj_video, 30, (width, height))
    train_every = 5
    steps = 0

    start_time = time.time()  # Start timer for total training time

    for episode in range(episodes):
        state, _ = env.reset()
        state = np.array(state, dtype=np.float32)
        episode_score = 0
        max_steps = 400  

        for step in range(max_steps):
            action = agent.select_action(state)
            step_result = env.step(action)
            next_state, reward, done = step_result[:3]
            info = step_result[3] if len(step_result) > 3 else None
            next_state = np.array(next_state, dtype=np.float32)
            episode_score += reward
            
            threshold = 200 - episode * 2  # Reduce el umbral con el progreso del episodio
            if done and episode_score < threshold:
                reward = -10
            
            agent.store_experience(state, action, reward, next_state, done)
            state = next_state

            if steps % train_every == 0:
                agent.train()

            if steps % target_update_steps == 0:
                agent.update_target_network()

            # frame = env.render()
            # if len(frame.shape) == 2:
            #     frame = np.expand_dims(frame, axis=-1)
            # frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # font = cv2.FONT_HERSHEY_SIMPLEX
            # text = f"Episodio: {episode + 1}"
            # position = (10, 30)
            # color = (0, 0, 0)
            # font_scale = 1
            # thickness = 2
            # cv2.putText(frame_bgr, text, position, font, font_scale, color, thickness, lineType=cv2.LINE_AA)

            # if steps % 10 == 0:
            #     Thread(target=save_frame, args=(frame_bgr, out)).start()

            if done:
                break

            steps += 1

        # Recopilar información para las gráficas
        agent.epsilon = max(agent.epsilon * agent.epsilon_decay, agent.epsilon_min)
        epsilons.append(agent.epsilon)
        scores.append(episode_score)

        print(f"Episodio {episode}, Puntaje: {episode_score}, Epsilon: {agent.epsilon:.2f}")

    total_training_time = time.time() - start_time  # Measure total training time
    print(f"Tiempo total de entrenamiento: {total_training_time:.2f} segundos")
    
    out.release()
    env.close()
    
    return scores, epsilons, total_training_time
    

In [4]:
all_scores = {}
all_epsilons = {}
training_times = {}


In [5]:
# Probar diferentes combinaciones de capas y neuronas, learning rate, gamma y batch_size
layer_configs_list = [
    [32, 32],  # 2 capas de 32 neuronas
    [64, 64],  # 2 capas de 64 neuronas
    [64, 128, 64],  # 3 capas de 64, 128, 64 neuronas
]

learning_rates = [0.001, 0.0001, 0.0005]
gammas = [0.95, 0.99, 0.995]
batch_sizes = [32, 64, 128]

for config in layer_configs_list:
    for lr in learning_rates:
        for gamma in gammas:
            for batch_size in batch_sizes:
                print(f"Entrenando con la arquitectura {config}, LR={lr}, Gamma={gamma}, BatchSize={batch_size}")
                scores, epsilons, total_time = train_dqn(layer_configs=config, learning_rate=lr, gamma=gamma, batch_size=batch_size)
                all_scores[f"{config}_LR{lr}_Gamma{gamma}_BatchSize{batch_size}"] = scores
                all_epsilons[f"{config}_LR{lr}_Gamma{gamma}_BatchSize{batch_size}"] = epsilons
                training_times[f"{config}_LR{lr}_Gamma{gamma}_BatchSize{batch_size}"] = [total_time] * len(scores)


Entrenando con la arquitectura [32, 32], LR=0.001, Gamma=0.95, BatchSize=32


C:\Users\samir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\samir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episodio 0, Puntaje: -400.0, Epsilon: 0.99
Episodio 1, Puntaje: -400.0, Epsilon: 0.99
Episodio 2, Puntaje: -400.0, Epsilon: 0.99
Episodio 3, Puntaje: -400.0, Epsilon: 0.98
Episodio 4, Puntaje: -400.0, Epsilon: 0.98
Episodio 5, Puntaje: -400.0, Epsilon: 0.97
Episodio 6, Puntaje: -400.0, Epsilon: 0.97
Episodio 7, Puntaje: -400.0, Epsilon: 0.96
Episodio 8, Puntaje: -400.0, Epsilon: 0.96
Episodio 9, Puntaje: -400.0, Epsilon: 0.95
Episodio 10, Puntaje: -400.0, Epsilon: 0.95
Episodio 11, Puntaje: -400.0, Epsilon: 0.94
Episodio 12, Puntaje: -400.0, Epsilon: 0.94
Episodio 13, Puntaje: -400.0, Epsilon: 0.93
Episodio 14, Puntaje: -400.0, Epsilon: 0.93
Tiempo total de entrenamiento: 203.71 segundos
Entrenando con la arquitectura [32, 32], LR=0.001, Gamma=0.95, BatchSize=64
Episodio 0, Puntaje: -400.0, Epsilon: 0.99
Episodio 1, Puntaje: -400.0, Epsilon: 0.99
Episodio 2, Puntaje: -400.0, Epsilon: 0.99
Episodio 3, Puntaje: -400.0, Epsilon: 0.98
Episodio 4, Puntaje: -400.0, Epsilon: 0.98
Episodio 5, 

In [6]:
# Para el desempeño:
def plot_performance(all_scores, layer_configs_list):
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12, 8))

    all_scores_flat = []
    all_episodes_flat = []
    labels_all = []

    # Extract scores and prepare labels
    for config, scores in all_scores.items():
        for episode, score in enumerate(scores):
            all_scores_flat.append(score)
            all_episodes_flat.append(episode)

    # Get the indices of the top 5 overall scores
    top_5_indices = sorted(range(len(all_scores_flat)), key=lambda i: all_scores_flat[i], reverse=True)[:5]
    top_5_labels = [labels_all[i] for i in top_5_indices]

    top_configs = list(set(top_5_labels))

    # Plot full scores for the top configurations
    for config in top_configs:
        episodes = list(range(len(all_scores[config])))
        scores = all_scores[config]
        plt.plot(episodes, scores, label=f"Config: {config}", marker='o', linestyle='-')

    plt.xlabel('Episodios')
    plt.ylabel('Puntuación por Episodio')
    plt.title('Top 5 Mejores Resultados en Desempeño a lo Largo de los Episodios')
    plt.legend()
    plt.show()


def plot_training_time(training_times):
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12, 8))

    # Almacenar todos los tiempos con sus configuraciones
    all_times_flat = []
    labels_all = []

    # Recorrer todas las configuraciones
    for config, time_taken in training_times.items():
        # Tomar el primer tiempo de cada lista de tiempos
        all_times_flat.append(time_taken[0])  # Solo tomamos el primer tiempo
        labels_all.append(config)  # Usamos la configuración como etiqueta

    # Ordenar todos los tiempos de menor a mayor y tomar los 5 mejores (menor tiempo es mejor)
    top_5_indices = sorted(range(len(all_times_flat)), key=lambda i: all_times_flat[i])[:5]
    top_5_labels = [labels_all[i] for i in top_5_indices]
    top_5_times = [all_times_flat[i] for i in top_5_indices]

    # Graficar las curvas completas para las configuraciones seleccionadas
    for config in top_5_labels:
        episodes = list(range(len(training_times[config])))  # Asumimos que todas las configuraciones tienen el mismo número de episodios
        times = training_times[config]
        plt.plot(episodes, times, label=f"Config: {config}", marker='o', linestyle='-')

    plt.xlabel('Episodios')
    plt.ylabel('Tiempo de Entrenamiento (segundos)')
    plt.title('Top 5 Mejores Resultados en Tiempo de Entrenamiento por Episodio')
    plt.legend()
    plt.show()

def plot_exploration_vs_exploitation(all_epsilons):
    plt.figure(figsize=(12, 8))

    # Calcular el "mejor" valor de epsilon para cada configuración, por ejemplo el máximo epsilon
    best_configs = sorted(all_epsilons.items(), key=lambda item: max(item[1]), reverse=True)[:5]
    
    # Graficar solo las 5 mejores configuraciones
    for config, epsilons in best_configs:
        episodes = list(range(len(epsilons)))  # Lista de episodios
        plt.plot(episodes, epsilons, label=f"Config: {config}", marker='o', linestyle='-')

    # Configuración del gráfico
    plt.xlabel('Episodios')
    plt.ylabel('Valor de Epsilon (Exploración)')
    plt.title('Top 5 Mejores Configuraciones de Exploración vs Explotación')
    plt.legend()
    plt.grid(True)
    plt.show()

In [10]:
# Graficar los resultados
plot_performance(all_scores, layer_configs_list)
plot_training_time(training_times)
plot_exploration_vs_exploitation(all_epsilons)

IndexError: list index out of range

<Figure size 1200x800 with 0 Axes>

In [9]:
layer_configs_list = [ 
    [64, 128, 64],  # 3 capas de 64, 128, 64 neuronas
]

learning_rates = [0.0005]
gammas = [0.995]
batch_sizes = [64]

for config in layer_configs_list:
    for lr in learning_rates:
        for gamma in gammas:
            for batch_size in batch_sizes:
                print(f"Entrenando con la arquitectura {config}, LR={lr}, Gamma={gamma}, BatchSize={batch_size}")
                scores, epsilons, total_time = train_dqn(layer_configs=config, learning_rate=lr, gamma=gamma, batch_size=batch_size)
                all_scores[f"{config}_LR{lr}_Gamma{gamma}_BatchSize{batch_size}"] = scores
                all_epsilons[f"{config}_LR{lr}_Gamma{gamma}_BatchSize{batch_size}"] = epsilons
                training_times[f"{config}_LR{lr}_Gamma{gamma}_BatchSize{batch_size}"] = [total_time] * len(scores)


Entrenando con la arquitectura [64, 128, 64], LR=0.0005, Gamma=0.995, BatchSize=64
Episodio 0, Puntaje: -400.0, Epsilon: 0.99
Episodio 1, Puntaje: -400.0, Epsilon: 0.99
Episodio 2, Puntaje: -400.0, Epsilon: 0.99
Episodio 3, Puntaje: -400.0, Epsilon: 0.98
Episodio 4, Puntaje: -400.0, Epsilon: 0.98
Episodio 5, Puntaje: -400.0, Epsilon: 0.97
Episodio 6, Puntaje: -400.0, Epsilon: 0.97
Episodio 7, Puntaje: -400.0, Epsilon: 0.96
Episodio 8, Puntaje: -400.0, Epsilon: 0.96
Episodio 9, Puntaje: -400.0, Epsilon: 0.95
Episodio 10, Puntaje: -400.0, Epsilon: 0.95
Episodio 11, Puntaje: -400.0, Epsilon: 0.94
Episodio 12, Puntaje: -400.0, Epsilon: 0.94
Episodio 13, Puntaje: -400.0, Epsilon: 0.93
Episodio 14, Puntaje: -400.0, Epsilon: 0.93
Tiempo total de entrenamiento: 209.93 segundos
